In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [2]:
%load_ext autoreload
%autoreload 2 

project_dir = os.path.dirname(os.getcwd())
print('project direcotry is ', project_dir)
if project_dir not in sys.path:
    sys.path.append(project_dir)
print(sys.path)

project direcotry is  /home/yuan/bio/antibody-antigen-complex
['/home/yuan/anaconda3/envs/openfold-env/lib/python310.zip', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/lib-dynload', '', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages/openfold-2.2.0-py3.10-linux-x86_64.egg', '/home/yuan/bio/antibody-antigen-complex']


## view structure

In [3]:
# get *pdb
from src.download import Download

pdb_id = '1F3R'
pdb_file = Download.download_pdb(pdb_id)

../pdb/1F3R.pdb is ready.


In [4]:
import py3Dmol
# Create a view
view = py3Dmol.view(width=400, height=300)
view.addModel(open(pdb_file).read(), 'pdb')
view.setStyle({'cartoon': {'color': 'spectrum'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## parse complex

In [5]:
# get chains
from src.parse_abag import ParseAbAg

outputs_dir = '../outputs'
p = ParseAbAg(pdb_file, outputs_dir)
# p.structure.header

# get chains
chains = p.get_chains()
pprint(chains)

Successfully retrieve structure of 1F3R
outputs dir:  ../outputs/1F/1F3R
[{'chains': [{'AA residues': 10, 'chain_id': 'A', 'seq': 'WNPGDYGGI'},
             {'AA residues': 257,
              'chain_id': 'B',
              'seq': 'QVQLLESGPGLVRPSETLSLTCTVSGFSLTSFSVSWVRHPSGKGPEWMGRMWYDGYTAYNSALKSRLSISRDTSKNQVFLKMNSLQTDDTGTYYCTRDLYGGYPLGFWYFDFWGPGTMVTVSSGGGGSGGGGSGGGGSDIKLTQSPSLLSASVGDRVTLSCKGSQNINNYLAWYQQKLGEAPKLLIYNTNSLQTGIPSRFSGSGSGTDYTLTISSLQPEDVATYFCYQYNNGYTFGAGTKLELKAAEQKLISEEDLN'}],
  'model_id': 0}]


In [6]:
type(p.structure.header['resolution'])

NoneType

In [7]:
p.structure.header['compound']

{'1': {'misc': '',
  'molecule': 'acetylcholine receptor alpha',
  'chain': 'a',
  'fragment': 'main immunogenic region (91-100)',
  'engineered': 'yes',
  'mutation': 'yes'},
 '2': {'misc': '',
  'molecule': 'fv antibody fragment',
  'chain': 'b',
  'fragment': 'fragment of mab198 raised against human acetylcholinereceptor '}}

In [8]:
# filter: is antibody-antigen complex
p.filter_antibody_antigen()
p.update_chains('chain_type')
print(p.structure_id)
pprint(p.info)

Pairwise chains of 1F3R are updated.
1F3R
[{'A': 'antigen', 'B': 'single'}]


In [9]:
# split chains
p.chains_to_pdb()

Saved chain ['B'] to ../outputs/1F/1F3R/1F3R_B.pdb
Saved chain ['A'] to ../outputs/1F/1F3R/1F3R_A.pdb
Saved chain ['B', 'A'] to ../outputs/1F/1F3R/1F3R_B_A.pdb


In [10]:
p.bounded

[{'type': 'single-antigen',
  'chains': [['B'], ['A'], ['B', 'A']],
  'pdb_files': ['../outputs/1F/1F3R/1F3R_B.pdb',
   '../outputs/1F/1F3R/1F3R_A.pdb',
   '../outputs/1F/1F3R/1F3R_B_A.pdb']}]

In [11]:
p.save_annot()

https://www.ebi.ac.uk/pdbe/api/mappings/1f3r
Save annotation of 1F3R: ../outputs/1F/1F3R/meta.json


## SASA

In [12]:
# calculate SASA
sasa_file = p.build_freesasa()

In [13]:
from src.analyze_sasa import AnalyzeSasa

p2= AnalyzeSasa(sasa_file)
p2.print_meta()
print(list(p2.data[0]['delta_total_sasa']))

key1 ['type', 'chains', 'pdb_files', 'sasa_files', 'delta_total_sasa']
single chain B ~ antigen chain A
------------------------------
['delta1', 'delta1_file', 'delta2', 'delta2_file']


In [14]:
# retrieve sequences
motifs = p2.retrieve_seq()
print('output file: ', motifs[1])
motifs

output file:  ../outputs/1F/1F3R/seq_sasa_delta-0_span-30.p


([('single',
   [{'seq': 'WMGRMWYDGYTAYNSALK',
     'sig_res': 10,
     'start': (46, '47'),
     'end': (63, '64'),
     'delta_sasa_cutoff': 0,
     'max_span': 30},
    {'seq': 'PLGFWY',
     'sig_res': 5,
     'start': (103, '104'),
     'end': (108, '109'),
     'delta_sasa_cutoff': 0,
     'max_span': 30},
    {'seq': 'YLAWYQQKLGEAPKLLIYN',
     'sig_res': 2,
     'start': (169, '170'),
     'end': (187, '188'),
     'delta_sasa_cutoff': 0,
     'max_span': 30},
    {'seq': 'YNNGYT',
     'sig_res': 6,
     'start': (228, '229'),
     'end': (233, '234'),
     'delta_sasa_cutoff': 0,
     'max_span': 30}],
   'antigen',
   [{'seq': 'WNPGDYGGI',
     'sig_res': 9,
     'start': (0, '67'),
     'end': (8, '75'),
     'delta_sasa_cutoff': 0,
     'max_span': 30}])],
 '../outputs/1F/1F3R/seq_sasa_delta-0_span-30.p')

In [15]:
# dataset for ML
seq_sasa_file = motifs[1]
AnalyzeSasa(seq_sasa_file).build_dataset()

[{'input': 'WNPGDYGGI',
  'input_label': 'antigen',
  'output': 'WMGRMWYDGYTAYNSALK | PLGFWY | YLAWYQQKLGEAPKLLIYN | YNNGYT',
  'output_label': 'single'}]

In [16]:
# complex
p2.print_delta_sasa(0)

complex: single chain B ~ antigen chain A
------------------------------
single chain:
    chain_id res_no  res aa      value
46         B     47  TRP  W  15.057930
49         B     50  ARG  R  26.901282
51         B     52  TRP  W  39.055501
55         B     56  TYR  Y  53.945645
56         B     57  THR  T   0.999328
57         B     58  ALA  A  40.558212
58         B     59  TYR  Y  16.313260
59         B     60  ASN  N   8.701156
60         B     61  SER  S  34.609343
63         B     64  LYS  K  28.634436
103        B    104  PRO  P  18.868657
104        B    105  LEU  L   2.703881
105        B    106  GLY  G   9.671883
106        B    107  PHE  F  35.371380
108        B    109  TYR  Y  12.788444
169        B    170  TYR  Y  45.074491
187        B    188  ASN  N   1.998656
228        B    229  TYR  Y  15.523589
229        B    230  ASN  N  22.043726
230        B    231  ASN  N  24.336486
231        B    232  GLY  G  27.038808
232        B    233  TYR  Y  72.266458
233        B    

## distance

In [17]:
# calculate distance
dist_file = p.build_dist()

In [18]:
from src.analyze_dist import AnalyzeDist

max_span = 30
p3= AnalyzeDist(dist_file, max_span=max_span)
p3.print_meta()
print(list(p3.data[0]['dist']))

key1 ['type', 'chains', 'pdb_files', 'dist']
single chain B ~ antigen chain A
------------------------------
['dist1', 'dist1_file', 'dist2', 'dist2_file']


In [19]:
# retrieve sequences
motifs = p3.retrieve_seq()
motifs

([('single',
   [{'seq': 'WMGRMWYDGYTAYNSALK',
     'sig_res': 10,
     'start': (46, 47),
     'end': (63, 64),
     'dist_cutoff': 5,
     'max_span': 30},
    {'seq': 'YGGYPLGFWY',
     'sig_res': 6,
     'start': (99, 100),
     'end': (108, 109),
     'dist_cutoff': 5,
     'max_span': 30},
    {'seq': 'YLAWYQQKLGEAPKLLIYN',
     'sig_res': 2,
     'start': (169, 170),
     'end': (187, 188),
     'dist_cutoff': 5,
     'max_span': 30},
    {'seq': 'YQYNNGYTFGAGTKLELKAAEQKLISEEDLN',
     'sig_res': 8,
     'start': (226, 227),
     'end': (256, 257),
     'dist_cutoff': 5,
     'max_span': 30}],
   'antigen',
   [{'seq': 'WNPGDYGGI',
     'sig_res': 9,
     'start': (0, 67),
     'end': (8, 75),
     'dist_cutoff': 5,
     'max_span': 30}])],
 '../outputs/1F/1F3R/seq_dist_dist-5_span-30.p')

In [20]:
seq_dist_file = motifs[1]
AnalyzeDist(seq_dist_file).build_dataset()

[{'input': 'WNPGDYGGI',
  'input_label': 'antigen',
  'output': 'WMGRMWYDGYTAYNSALK | YGGYPLGFWY | YLAWYQQKLGEAPKLLIYN | YQYNNGYTFGAGTKLELKAAEQKLISEEDLN',
  'output_label': 'single'}]

In [21]:
# complex
p3.print_min_dist(0)

------------------------------
     res_no  res atom      value aa
0         1  GLN    C  23.561633  Q
1         2  VAL    C  18.013430  V
2         3  GLN    C  20.778929  Q
3         4  LEU    C  14.522016  L
4         5  LEU    C  20.367520  L
..      ...  ...  ...        ... ..
252     253  GLU    C  16.214252  E
253     254  GLU    C  13.588741  E
254     255  ASP    C   9.624752  D
255     256  LEU    C   7.030269  L
256     257  ASN    C   4.683882  N

[257 rows x 5 columns]
------------------------------
   res_no  res atom     value aa
0      67  TRP    C  1.894786  W
1      68  ASN    C  2.225700  N
2      69  PRO    C  3.121428  P
3      70  GLY    C  1.786383  G
4      71  ASP    C  1.888589  D
5      72  TYR    C  1.695133  Y
6      73  GLY    C  2.701292  G
7      74  GLY    C  2.089406  G
8      75  ILE    C  1.774871  I
